# Take snapshots of time-series for each file and save them to disk

In [2]:
%reset -f
import os
import matplotlib.pyplot as plt
import numpy as np

from data import DataLoader, find_segs
from db import make_session, d_models
from plot import shade, set_font_size

import CONFIG as C


SAVE_DIR = 'data_snapshots'
LABELS = ('behav', 'gcamp')
BEHAV_VARIABLES = [
    ['speed'], ['ball'], ['v_ang'], ['air']
]
GCAMP_VARIABLES = [
    ['g2s'], ['g3s'], ['g4s'], ['g5s'], ['g2d'], ['g3d'], ['g4d'], ['g5d'],
]

In [3]:
session = make_session()
trials = session.query(d_models.Trial).all()
session.close()

for ctr, trial in enumerate(trials):
    print('Loading data for trial {}/{} ({})'.format(ctr+1, len(trials), trial.name))
    trials[ctr].dl = DataLoader(trial, sfx='0', vel_filt=None)

Loading data for trial 1/152 (20160904.Fly4.2)
Loading clean data from file "clean_0.csv"...
Loading data for trial 2/152 (20160904.Fly4.3)
Loading clean data from file "clean_0.csv"...
Loading data for trial 3/152 (20160904.Fly4.6)
Loading clean data from file "clean_0.csv"...
Loading data for trial 4/152 (20160904.Fly4.7)
Loading clean data from file "clean_0.csv"...
Loading data for trial 5/152 (20160904.Fly5.3)
Loading clean data from file "clean_0.csv"...
Loading data for trial 6/152 (20160904.Fly5.4)
Loading clean data from file "clean_0.csv"...
Loading data for trial 7/152 (20160904.Fly6.1)
Loading clean data from file "clean_0.csv"...
Loading data for trial 8/152 (20160904.Fly6.2)
Loading clean data from file "clean_0.csv"...
Loading data for trial 9/152 (20160904.Fly6.3)
Loading clean data from file "clean_0.csv"...
Loading data for trial 10/152 (20160904.Fly6.4)
Loading clean data from file "clean_0.csv"...
Loading data for trial 11/152 (20160904.Fly6.7)
Loading clean data fr

Loading data for trial 97/152 (20170201.Fly4.2)
Loading clean data from file "clean_0.csv"...
Loading data for trial 98/152 (20170201.Fly4.3)
Loading clean data from file "clean_0.csv"...
Loading data for trial 99/152 (20170111.Fly2.5)
Loading clean data from file "clean_0.csv"...
Loading data for trial 100/152 (20170310.Fly2.6)
Loading clean data from file "clean_0.csv"...
Loading data for trial 101/152 (20170310.Fly3.1)
Loading clean data from file "clean_0.csv"...
Loading data for trial 102/152 (20170310.Fly3.2)
Loading clean data from file "clean_0.csv"...
Loading data for trial 103/152 (20170313.Fly1.1)
Loading clean data from file "clean_0.csv"...
Loading data for trial 104/152 (20170313.Fly1.2)
Loading clean data from file "clean_0.csv"...
Loading data for trial 105/152 (20170313.Fly1.3)
Loading clean data from file "clean_0.csv"...
Loading data for trial 106/152 (20170313.Fly1.4)
Loading clean data from file "clean_0.csv"...
Loading data for trial 107/152 (20170313.Fly1.8)
Load

### Define function that saves data time-series to PNG

In [4]:
def trial_to_png(
    save_file, trial, variables, width=60, ax_height=3,
    lw=2, colors=None, y_lims=None, y_tick_spacings=None,
    shading=None):
    """
    Save the complete time-series data for several quantities for
    a trial to a single PNG image for efficient viewing.
    
    :param shading: shading to apply on top of time-series traces; dict
        where keys are rgbas and vals are segments to shade
    """
    if colors is None:
        colors = {}
    if y_lims is None:
        y_lims = {}
    if y_tick_spacings is None:
        y_tick_spacings = {}
    if shading is None:
        shading == {}
    
    # make sure each element of variables is a list of variables
    variables = [vs if hasattr(vs, '__iter__') else [vs] for vs in variables]
        
    # make sure save directory exists
    save_dir = os.path.dirname(save_file)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    # get quantities from trial
    ts = trial.dl.t
    unique = list(set(sum(variables, [])))
    for v in unique:
        if v not in colors:
            colors[v] = 'k'
            
    data = {v: getattr(trial.dl, v) for v in unique}
    
    # arrange figure
    fig_size = (width, ax_height*len(variables))
    fig, axs = plt.subplots(
        len(variables), 1, figsize=fig_size, tight_layout=True, squeeze=False)
    
    for ax, vs in zip(axs[:, 0], variables):
        
        ax.set_xlabel('t (s)')
        
        ax.plot(ts, data[vs[0]], color=colors[vs[0]], lw=lw)
        ax.set_ylabel(vs[0], color=colors[vs[0]])
        
        ax.set_xlim(ts[0], ts[-1])
        ax.grid()
        
        if vs[0] in y_lims:
            ax.set_ylim(y_lims[vs[0]])
        
        if vs[0] in y_tick_spacings:
            ax.set_yticks(np.arange(
                *ax.get_ylim(), y_tick_spacings[vs[0]]))
            
        if shading is not None:
            for rgba, segs in shading.items():
                shade(ax, segs, rgba)
            
        set_font_size(ax, 16)
        
        if len(vs) == 1:
            continue
            
        ax_twin = ax.twinx()
        
        # plot all remaining variables on second axis
        for v in vs[1:]:
            ax_twin.plot(ts, data[v], color=colors[v], lw=lw)
            
            if v in y_lims:
                ax_twin.set_ylim(y_lims[v])

            if v in y_tick_spacings:
                ax_twin.set_yticks(np.arange(
                    *ax_twin.get_ylim(), y_tick_spacings[v]))
        
        ax_twin.set_xlim(ts[0], ts[-1])
        ax_twin.set_ylabel(', '.join(vs[1:]))
        
        ax_twin.set_title(', '.join(vs))
        
        set_font_size(ax_twin, 16)
        
    # save png
    fig.savefig(save_file)
    plt.close(fig)
    
    return save_file

### Basic behavioral and gcamp variables

In [8]:
for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    for label, vs in zip(LABELS, [BEHAV_VARIABLES, GCAMP_VARIABLES]):
        
        save_path = os.path.join(save_dir, '{}.png'.format(label))
        trial_to_png(save_path, trial, vs, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
        print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "d

Snapshot saved at "data_snapshots/20160905.Fly6/20160905.Fly6.3 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly6.4"...
Snapshot saved at "data_snapshots/20160905.Fly6/20160905.Fly6.4 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly6/20160905.Fly6.4 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly7.1"...
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.1 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.1 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly7.2"...
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly7.3"...
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.3 (closed_

Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.3"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.3 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.4"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.4 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.4 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.5"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.5 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.5 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.6"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.6 (closed_

Snapshot saved at "data_snapshots/20170313.Fly1/20170313.Fly1.8 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170313.Fly3.1"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.1 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.1 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170313.Fly3.2"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170313.Fly3.3"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170330.Fly1.1"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.1 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.1 (closed_

Snapshot saved at "data_snapshots/20170310.Fly3/20170310.Fly3.5 (no_air)/gcamp.png".
Saving data snapshot for trial "20170313.Fly1.7"...
Snapshot saved at "data_snapshots/20170313.Fly1/20170313.Fly1.7 (no_air)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly1/20170313.Fly1.7 (no_air)/gcamp.png".
Saving data snapshot for trial "20170313.Fly3.6"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.6 (no_air)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.6 (no_air)/gcamp.png".
Saving data snapshot for trial "20170330.Fly1.3"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.3 (no_air)/behav.png".
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.3 (no_air)/gcamp.png".
Saving data snapshot for trial "20170331.Fly1.3"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.3 (no_air)/behav.png".
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.3 (no_air)/gcamp.png".
Saving data snapshot for tr

### Close up of walking speed

In [6]:
session = make_session()
trials = session.query(d_models.TrialNew).all()
session.close()

for trial in trials:
    trial.dl = DataLoader(trial, sfx='0', vel_filt=None)
    
    if trial.walking_threshold is not None:
        states = trial.dl.state
        ts_ = np.concatenate([
            trial.dl.t,
            [trial.dl.t[-1] + C.DT]
        ])
        rgbas = [(0, 0, 0, 0.2), (1, 0, 0, 0.2), (0, 1, 0, 0.2)]
        labels = ['A', 'P', 'W']
        
        shading = {}
        for rgba, label in zip(rgbas, labels):
            segs_idx = find_segs(states == label)
            segs = np.nan * np.ones(segs_idx.shape)
            segs[:, 0] = ts_[segs_idx[:, 0]]
            segs[:, 1] = ts_[segs_idx[:, 1]]
            
            shading[rgba] = segs
    else:
        shading=None
    
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'speed.png')
    trial_to_png(
        save_path, trial, [['speed']], width=60, ax_height=8,
        y_lims={'speed': [0, None]}, y_tick_spacings={'speed': 0.005},
        shading=shading)

    print('Snapshot saved at "{}".'.format(save_path))

Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data sn

Snapshot saved at "data_snapshots/20160905.Fly8/20160905.Fly8.3 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160905.Fly8.4"...
Snapshot saved at "data_snapshots/20160905.Fly8/20160905.Fly8.4 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160905.Fly8.5"...
Snapshot saved at "data_snapshots/20160905.Fly8/20160905.Fly8.5 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160906.Fly1.1"...
Snapshot saved at "data_snapshots/20160906.Fly1/20160906.Fly1.1 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160906.Fly1.2"...
Snapshot saved at "data_snapshots/20160906.Fly1/20160906.Fly1.2 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20160906.Fly1.3"...
Snapshot saved at "data_snapshots/20160906.Fly1/20160906.Fly

Snapshot saved at "data_snapshots/20170131.Fly2/20170131.Fly2.2 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170131.Fly2.8"...
Snapshot saved at "data_snapshots/20170131.Fly2/20170131.Fly2.8 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170201.Fly1.1"...
Snapshot saved at "data_snapshots/20170201.Fly1/20170201.Fly1.1 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170201.Fly2.1"...
Snapshot saved at "data_snapshots/20170201.Fly2/20170201.Fly2.1 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170201.Fly2.2"...
Snapshot saved at "data_snapshots/20170201.Fly2/20170201.Fly2.2 (closed_loop)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170201.Fly2.3"...
Snapshot saved at "data_snapshots/20170201.Fly2/20170201.Fly

Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.5 (driven_sinusoidal)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170117.Fly1.6"...
Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.6 (driven_sinusoidal)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170118.Fly2.4"...
Snapshot saved at "data_snapshots/20170118.Fly2/20170118.Fly2.4 (driven_sinusoidal)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170118.Fly2.5"...
Snapshot saved at "data_snapshots/20170118.Fly2/20170118.Fly2.5 (driven_sinusoidal)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170118.Fly2.6"...
Snapshot saved at "data_snapshots/20170118.Fly2/20170118.Fly2.6 (driven_sinusoidal)/speed.png".
Loading clean data from file "clean_0.csv"...
Saving data snapshot for trial "20170118.Fly2.7"...
Snapshot saved at "data_snapsh

### Speed overlaid with GS

In [9]:
vs = [
    ['speed', 'g2s'],
    ['speed', 'g3s'],
    ['speed', 'g4s'],
    ['speed', 'g5s'],
]
colors = {'speed': 'k', 'g2s': 'm', 'g3s': 'm', 'g4s': 'm', 'g5s': 'm'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'speed_vs_gs.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_snapshots/20160904.Fly6/20160904.Fly

Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160906.Fly6.2"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.2 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160906.Fly6.3"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.3 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20160906.Fly6.4"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.4 (closed_loop)/speed_vs_gs.png".
Saving data snap

Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20170331.Fly1.2"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20170331.Fly2.4"...
Snapshot saved at "data_snapshots/20170331.Fly2/20170331.Fly2.4 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20170331.Fly3.2"...
Snapshot saved at "data_snapshots/20170331.Fly3/20170331.Fly3.2 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20170428.Fly1.1"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.1 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20170428.Fly1.2"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.2 (closed_loop)/speed_vs_gs.png".
Saving data snapshot for trial "20170428.Fly1.5"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.5 (driven_random)/speed_vs_gs.png".
Saving data sn

### Angular velocity overlaid with GD

In [10]:
vs = [
    ['v_ang', 'g2d'],
    ['v_ang', 'g3d'],
    ['v_ang', 'g4d'],
    ['v_ang', 'g5d'],
]
colors = {'v_ang': 'k', 'g2d': 'g', 'g3d': 'g', 'g4d': 'g', 'g5d': 'g'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'v_ang_vs_gd.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_snapshots/20160904.Fly6/20160904.Fly

Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.2"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.2 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.3"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.3 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.4"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.4 (closed_loop)/v_ang_vs_gd.png".
Saving data snap

Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20170331.Fly1.2"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20170331.Fly2.4"...
Snapshot saved at "data_snapshots/20170331.Fly2/20170331.Fly2.4 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20170331.Fly3.2"...
Snapshot saved at "data_snapshots/20170331.Fly3/20170331.Fly3.2 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.1"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.1 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.2"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.2 (closed_loop)/v_ang_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.5"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.5 (driven_random)/v_ang_vs_gd.png".
Saving data sn

### Angular velocity overlaid with ddt GD

In [10]:
vs = [
    ['v_ang', 'ddt_g2d'],
    ['v_ang', 'ddt_g3d'],
    ['v_ang', 'ddt_g4d'],
    ['v_ang', 'ddt_g5d'],
]
colors = {'v_ang': 'k', 'ddt_g2d': 'g', 'ddt_g3d': 'g', 'ddt_g4d': 'g', 'ddt_g5d': 'g'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'v_ang_vs_ddt_gd.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_snapshots/20

Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.1 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly3.2"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.2 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly3.3"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly6.2"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.2 (closed_loop)/v_ang_vs_d

Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20170313.Fly3.3"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20170330.Fly1.1"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.1 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20170330.Fly1.2"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.2 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20170331.Fly1.1"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20170331.Fly1.2"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop)/v_ang_vs_ddt_gd.png".
Saving data snapshot for trial "20170331.Fly2.4"...
Snapshot saved at "data_snapshots/20170331.Fly2/20170331.Fly2.4 (closed_loop)/v_ang_vs_d

### Angular velocity overlaid with d2dt2 GD

In [15]:
vs = [
    ['v_ang', 'ddt_ddt_g2d'],
    ['v_ang', 'ddt_ddt_g3d'],
    ['v_ang', 'ddt_ddt_g4d'],
    ['v_ang', 'ddt_ddt_g5d'],
]
colors = {'v_ang': 'k', 'ddt_ddt_g2d': 'g', 'ddt_ddt_g3d': 'g', 'ddt_ddt_g4d': 'g', 'ddt_ddt_g5d': 'g'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'v_ang_vs_d2dt2_gd.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_

Snapshot saved at "data_snapshots/20160906.Fly1/20160906.Fly1.7 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly3.1"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.1 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly3.2"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.2 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly3.3"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop

Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.1 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170313.Fly3.2"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170313.Fly3.3"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170330.Fly1.1"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.1 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170330.Fly1.2"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.2 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170331.Fly1.1"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (closed_loop)/v_ang_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170331.Fly1.2"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop

### Air tube velocity overlaid with GD

In [11]:
vs = [
    ['v_air', 'g2d'],
    ['v_air', 'g3d'],
    ['v_air', 'g4d'],
    ['v_air', 'g5d'],
]
colors = {'v_ang': 'k', 'g2d': 'g', 'g3d': 'g', 'g4d': 'g', 'g5d': 'g'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'v_air_vs_gd.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_snapshots/20160904.Fly6/20160904.Fly

Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.2"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.2 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.3"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.3 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.4"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.4 (closed_loop)/v_air_vs_gd.png".
Saving data snap

Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20170331.Fly1.2"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20170331.Fly2.4"...
Snapshot saved at "data_snapshots/20170331.Fly2/20170331.Fly2.4 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20170331.Fly3.2"...
Snapshot saved at "data_snapshots/20170331.Fly3/20170331.Fly3.2 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.1"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.1 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.2"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.2 (closed_loop)/v_air_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.5"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.5 (driven_random)/v_air_vs_gd.png".
Saving data sn

### Air tube position overlaid with GD

In [5]:
vs = [
    ['air', 'g2d'],
    ['air', 'g3d'],
    ['air', 'g4d'],
    ['air', 'g5d'],
]
colors = {'v_ang': 'k', 'g2d': 'g', 'g3d': 'g', 'g4d': 'g', 'g5d': 'g'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'air_vs_gd.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_snapshots/20160904.Fly6/20160904.Fly6.1 (closed_

Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.2"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.2 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.3"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.3 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20160906.Fly6.4"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.4 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial

Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20170331.Fly2.4"...
Snapshot saved at "data_snapshots/20170331.Fly2/20170331.Fly2.4 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20170331.Fly3.2"...
Snapshot saved at "data_snapshots/20170331.Fly3/20170331.Fly3.2 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.1"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.1 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.2"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.2 (closed_loop)/air_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.5"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.5 (driven_random)/air_vs_gd.png".
Saving data snapshot for trial "20170428.Fly1.7"...
Snapshot saved at "data_snapshots/20170428.Fly1/20170428.Fly1.7 (closed_loop)/air_vs_gd.png".
Saving data snapshot for tri

### Air tube velocity overlaid with ddt GD

In [11]:
vs = [
    ['v_air', 'ddt_g2d'],
    ['v_air', 'ddt_g3d'],
    ['v_air', 'ddt_g4d'],
    ['v_air', 'ddt_g5d'],
]
colors = {'v_ang': 'k', 'ddt_g2d': 'g', 'ddt_g3d': 'g', 'ddt_g4d': 'g', 'ddt_g5d': 'g'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'v_air_vs_ddt_gd.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_snapshots/20

Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.1 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly3.2"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.2 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly3.3"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20160906.Fly6.2"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.2 (closed_loop)/v_air_vs_d

Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20170313.Fly3.3"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20170330.Fly1.1"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.1 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20170330.Fly1.2"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.2 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20170331.Fly1.1"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20170331.Fly1.2"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop)/v_air_vs_ddt_gd.png".
Saving data snapshot for trial "20170331.Fly2.4"...
Snapshot saved at "data_snapshots/20170331.Fly2/20170331.Fly2.4 (closed_loop)/v_air_vs_d

### Air tube velocity overlaid with d2dt2 GD

In [14]:
vs = [
    ['v_air', 'ddt_ddt_g2d'],
    ['v_air', 'ddt_ddt_g3d'],
    ['v_air', 'ddt_ddt_g4d'],
    ['v_air', 'ddt_ddt_g5d'],
]
colors = {'v_ang': 'k', 'ddt_ddt_g2d': 'g', 'ddt_ddt_g3d': 'g', 'ddt_ddt_g4d': 'g', 'ddt_ddt_g5d': 'g'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'v_air_vs_d2dt2_gd.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at "data_

Snapshot saved at "data_snapshots/20160906.Fly1/20160906.Fly1.7 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly3.1"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.1 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly3.2"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.2 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly3.3"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (closed_loop

Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.1 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170313.Fly3.2"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170313.Fly3.3"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170330.Fly1.1"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.1 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170330.Fly1.2"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.2 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170331.Fly1.1"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (closed_loop)/v_air_vs_d2dt2_gd.png".
Saving data snapshot for trial "20170331.Fly1.2"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.2 (closed_loop

### Paired GSs overlaid with speed

In [7]:
vs = [
    ['speed', 'g2s', 'g3s'],
    ['speed', 'g2s', 'g4s'],
    ['speed', 'g2s', 'g5s'],
    ['speed', 'g3s', 'g4s'],
    ['speed', 'g3s', 'g5s'],
    ['speed', 'g4s', 'g5s'],
]
colors = {'speed': (.5, .5, .5, .5), 'g2s': 'm', 'g3s': 'b', 'g4s': 'g', 'g5s': 'c'}

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    save_path = os.path.join(save_dir, 'speed_vs_gs_paired.png')
    trial_to_png(save_path, trial, variables=vs, colors=colors, shading={(.5, .5, .5, .3): [(C.ODOR_START, C.ODOR_END)]})
        
    print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.3 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160904.Fly5.4"...
Snapshot saved at "data_snapshots/20160904.Fly5/20160904.Fly5.4 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160904.Fly6.1"...
Snapshot saved at 

Snapshot saved at "data_snapshots/20160906.Fly1/20160906.Fly1.7 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160906.Fly3.1"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.1 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160906.Fly3.2"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.2 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160906.Fly3.3"...
Snapshot saved at "data_snapshots/20160906.Fly3/20160906.Fly3.3 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160906.Fly4.1"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.1 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160906.Fly4.2"...
Snapshot saved at "data_snapshots/20160906.Fly4/20160906.Fly4.2 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20160906.Fly6.1"...
Snapshot saved at "data_snapshots/20160906.Fly6/20160906.Fly6.1 (close

Snapshot saved at "data_snapshots/20170313.Fly1/20170313.Fly1.8 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20170313.Fly3.1"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.1 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20170313.Fly3.2"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20170313.Fly3.3"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20170330.Fly1.1"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.1 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20170330.Fly1.2"...
Snapshot saved at "data_snapshots/20170330.Fly1/20170330.Fly1.2 (closed_loop)/speed_vs_gs_paired.png".
Saving data snapshot for trial "20170331.Fly1.1"...
Snapshot saved at "data_snapshots/20170331.Fly1/20170331.Fly1.1 (close

### Write js file listing all trials and image names

In [6]:
TRIAL_FILE = 'js_viewer/trials.js'

# get all files in save directory
sub_dirs = [
    os.path.join(SAVE_DIR, d) for d in os.listdir(SAVE_DIR)
    if os.path.isdir(os.path.join(SAVE_DIR, d))
    and not d.startswith('.')]

trial_keys = []
trial_paths = []

for sub_dir in sub_dirs:
    trial_keys_ = [
        d for d in os.listdir(sub_dir)
        if os.path.isdir(os.path.join(sub_dir, d))
        and not d.startswith('.')
    ]
    
    trial_paths_ = [os.path.join(sub_dir, tk) for tk in trial_keys_]
    
    trial_keys.extend(trial_keys_)
    trial_paths.extend(trial_paths_)

In [7]:
# open file
with open(TRIAL_FILE, 'w') as f:
    f.write('var trials = {};\n')
    
    for tk, tp in zip(trial_keys, trial_paths):
        f.write('trials["{}"] = "{}";\n'.format(tk, tp))
        
    f.write('\n')
    f.write('var plot_types = [\n')
    
    for plot_type in (
            'behav', 'gcamp', 'speed', 'speed_vs_gs', 'speed_vs_gs_paired',
            'v_ang_vs_gd', 'v_air_vs_gd', 'air_vs_gd', 'v_ang_vs_ddt_gd', 'v_air_vs_ddt_gd',
            'v_ang_vs_d2dt2_gd', 'v_air_vs_d2dt2_gd',
            'g4s_vs_speed', 'g4s_vs_speed_1'):
        f.write('  "{}",\n'.format(plot_type))
    f.write(']\n')